<h4> Import Dependencies </h4>

In [80]:
import os
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
from pprint import pprint
import sqlite3
import csv
from sqlalchemy import create_engine


<h2>Financial Data, API Exctraction</h2>

<h4>Make API Calls and Populate a DataFrame</h4>

In [81]:
#Starting Year
year = 2000

In [91]:
#API url path
url = f"https://api.open.fec.gov/v1/elections/?api_key=9SpGS9Og85LkLUFAfVSQYVmqnPIVbhHzzomCynhK&sort=-total_receipts&sort_hide_null=false&office=president&election_full=true&sort_null_only=false&cycle={year}&sort_nulls_last=false&page=1"

# read data from the API 
response = requests.get(url).json()
# pprint(response)

# convert JSON to dataframe
candidate_finance = pd.json_normalize(response, "results")
# print(candidate_finance.shape)

#Append following year data to existing DataFrame
for yr in range(year + 4, 2017, 4):
    url = f"https://api.open.fec.gov/v1/elections/?api_key=9SpGS9Og85LkLUFAfVSQYVmqnPIVbhHzzomCynhK&sort=-total_receipts&sort_hide_null=false&office=president&election_full=true&sort_null_only=false&cycle={yr}&sort_nulls_last=false&page=1"
    response = requests.get(url).json()
    sub_df = pd.json_normalize(response, "results")
    candidate_finance.head(2)
#     print(f"{yr} {type(sub_df)}  ================================")
    candidate_finance = candidate_finance.append(sub_df, ignore_index=True)
    
# Format dataframe
candidate_finance = candidate_finance[['candidate_election_year', 'candidate_id', 'candidate_name', 'party_full', 'total_receipts', 'total_disbursements', 'cash_on_hand_end_period']]
candidate_finance.style.format({"total_receipts": '${0:,.2f}',
                                "total_disbursements": '${0:,.2f}',
                                "cash_on_hand_end_period": '${0:,.2f}',})

candidate_finance["candidate_name"]

0                  BUSH, GEORGE W
1                        GORE, AL
2                   FORBES, STEVE
3                  MCCAIN, JOHN S
4                   BRADLEY, BILL
                 ...             
95        O'MALLEY, MARTIN JOSEPH
96              GRAHAM, LINDSEY O
97                 HUCKABEE, MIKE
98           SANTORUM, RICHARD J.
99    MCMULLIN, EVAN / MINDY FINN
Name: candidate_name, Length: 100, dtype: object

<h4>Clean Data</h4>

In [92]:
def rename(name):
    #Some names are in format: Pres / Vice Pres - we only want President name
    name = name.split("/")[0]
    #President name in Last, First MI. format - we want First MI Last
    name_components = name.split(",")
    new_name = name_components[-1] + " " + name_components[0]
    new_name = new_name.replace('.', '')
    new_name = new_name.replace('  ', ' ')    
    return new_name.upper().strip()

#Apply the renaming funtion to the dataframe
candidate_finance["candidate_name"] = candidate_finance["candidate_name"].apply(rename)
candidate_finance


,candidate_election_year,candidate_id,candidate_name,party_full,total_receipts,total_disbursements,cash_on_hand_end_period
0,2000,P00003335,GEORGE W BUSH,REPUBLICAN PARTY,1.936577e+08,1.864560e+08,7201734.18
1,2000,P80000912,AL GORE,DEMOCRATIC PARTY,1.331080e+08,1.203352e+08,12772827.00
2,2000,P60003852,STEVE FORBES,REPUBLICAN PARTY,8.604857e+07,8.605477e+07,0.00
3,2000,P80002801,JOHN S MCCAIN,REPUBLICAN PARTY,5.863623e+07,5.854997e+07,86262.00
4,2000,P80000516,BILL BRADLEY,DEMOCRATIC PARTY,5.115617e+07,5.042141e+07,775203.00
...,...,...,...,...,...,...,...
95,2016,P60007671,MARTIN JOSEPH O'MALLEY,DEMOCRATIC PARTY,6.377550e+06,6.285182e+06,92368.71
96,2016,P60007697,LINDSEY O GRAHAM,REPUBLICAN PARTY,6.010010e+06,6.010010e+06,0.00
97,2016,P80003478,MIKE HUCKABEE,REPUBLICAN PARTY,4.335975e+06,4.330418e+06,5557.58
98,2016,P20002721,RICHARD J SANTORUM,REPUBLICAN PARTY,2.296044e+06,2.312075e+06,110242.76


In [98]:
# add new column for key
candidate_finance["key"] = candidate_finance["candidate_name"] + candidate_finance["candidate_election_year"].astype(str)
candidate_finance["key"] = candidate_finance["key"].replace(" ", "", regex=True)
candidate_finance

,candidate_election_year,candidate_id,candidate_name,party_full,total_receipts,total_disbursements,cash_on_hand_end_period,key
0,2000,P00003335,GEORGE W BUSH,REPUBLICAN PARTY,1.936577e+08,1.864560e+08,7201734.18,GEORGEWBUSH2000
1,2000,P80000912,AL GORE,DEMOCRATIC PARTY,1.331080e+08,1.203352e+08,12772827.00,ALGORE2000
2,2000,P60003852,STEVE FORBES,REPUBLICAN PARTY,8.604857e+07,8.605477e+07,0.00,STEVEFORBES2000
3,2000,P80002801,JOHN S MCCAIN,REPUBLICAN PARTY,5.863623e+07,5.854997e+07,86262.00,JOHNSMCCAIN2000
4,2000,P80000516,BILL BRADLEY,DEMOCRATIC PARTY,5.115617e+07,5.042141e+07,775203.00,BILLBRADLEY2000
...,...,...,...,...,...,...,...,...
95,2016,P60007671,MARTIN JOSEPH O'MALLEY,DEMOCRATIC PARTY,6.377550e+06,6.285182e+06,92368.71,MARTINJOSEPHO'MALLEY2016
96,2016,P60007697,LINDSEY O GRAHAM,REPUBLICAN PARTY,6.010010e+06,6.010010e+06,0.00,LINDSEYOGRAHAM2016
97,2016,P80003478,MIKE HUCKABEE,REPUBLICAN PARTY,4.335975e+06,4.330418e+06,5557.58,MIKEHUCKABEE2016
98,2016,P20002721,RICHARD J SANTORUM,REPUBLICAN PARTY,2.296044e+06,2.312075e+06,110242.76,RICHARDJSANTORUM2016


<h4>Export Dataframe to CSV and JSON Formats</h4>

In [99]:
candidate_finance.to_csv("candidate_finance_2.csv")
candidate_finance.to_json("candidate_finance_2.json")

<h2>Election Result Data, CSV Extraction</h2>

<h4>Read CSV and Populate a DataFrame</h4>

In [100]:
election_csv = f"{year}_PopularVote.csv"

path = os.path.join("Resources", election_csv)
election_df = pd.read_csv(path, encoding="ISO-8859-1")

<h4>Clean Data</h4>

In [101]:
election_df = election_df.rename(columns = {
    "Candidate (Party Label)": "name"
})
election_df = election_df[["name", "Popular Vote Total", "Percent of Popular Vote"]]
election_df["year"] = 2000
election_df = election_df.dropna()

def conv_name(long_name):
    #Some data in FOrmat Name (Party) - we just want name
    short_name = long_name.split('(')[0]
    #MI is followed by ".", we want to remove that
    short_name = short_name.replace('.', '')
    #we want to make sure all name components are spaced correctly
    components = short_name.split(' ')
    final_name = ""
    for component in components:
        component = component.strip()
        if final_name == "":
            final_name = component
        else:
            final_name = final_name + " " + component
    return final_name.upper().strip()

#apply the name cleaning to the dataframe
election_df["name"] = election_df["name"].apply(conv_name)
election_df.head()

,name,Popular Vote Total,Percent of Popular Vote,year
1,AL GORE,"50,999,897",48.38,2000
2,GEORGE W BUSH,"50,456,002",47.87,2000
3,RALPH NADER,"2,882,955",2.74,2000
4,PATRICK J BUCHANAN,"448,895",0.42,2000
5,HARRY BROWNE,"384,431",0.36,2000


<h4>Append following year data to DataFrame</h4?

In [102]:
for yr in range(year + 4, 2017, 4):
    path = os.path.join("Resources", f"{yr}_PopularVote.csv")
    temp_df = pd.read_csv(path, encoding="ISO-8859-1")
    if yr == 2004: #2004 data is formatted differently
        temp_df = temp_df.rename(columns = {
            "Candidate": "name"
        })
        temp_df = temp_df[["name", "Popular Vote Total", "Percent of Popular Vote"]]
        temp_df = temp_df.dropna()
        temp_df["name"] = temp_df["name"].apply(conv_name)
        temp_df["year"] = yr
        election_df = election_df.append(temp_df, ignore_index=True)
    else:
        temp_df = temp_df.rename(columns = {
            "Candidate (Party Label)": "name"
        })
        temp_df = temp_df[["name", "Popular Vote Total", "Percent of Popular Vote"]]
        temp_df = temp_df.dropna()
        temp_df["name"] = temp_df["name"].apply(conv_name)
        temp_df["year"] = yr
        election_df = election_df.append(temp_df, ignore_index=True)

election_df = election_df.rename(columns = {
    "name": "name",
    "Popular Vote Total": "votes",
    "Percent of Popular Vote": "votepct"
})

election_df["votes"] = election_df["votes"].replace(",","", regex=True)
election_df["votepct"] = election_df["votepct"].replace("%","", regex=True)

election_df

,name,votes,votepct,year
0,AL GORE,50999897,48.38,2000
1,GEORGE W BUSH,50456002,47.87,2000
2,RALPH NADER,2882955,2.74,2000
3,PATRICK J BUCHANAN,448895,0.42,2000
4,HARRY BROWNE,384431,0.36,2000
...,...,...,...,...
121,ROD SILVA,751,0.00,2016
122,PRINCESS JACOB,749,0.00,2016
123,JERRY WHITE,475,0.00,2016
124,BRADFORD LYTTLE,382,0.00,2016


In [103]:
# add new column for key
election_df["key"] = election_df["name"] + election_df["year"].astype(str)
election_df["key"] = election_df["key"].replace(" ", "", regex=True)
election_df

,name,votes,votepct,year,key
0,AL GORE,50999897,48.38,2000,ALGORE2000
1,GEORGE W BUSH,50456002,47.87,2000,GEORGEWBUSH2000
2,RALPH NADER,2882955,2.74,2000,RALPHNADER2000
3,PATRICK J BUCHANAN,448895,0.42,2000,PATRICKJBUCHANAN2000
4,HARRY BROWNE,384431,0.36,2000,HARRYBROWNE2000
...,...,...,...,...,...
121,ROD SILVA,751,0.00,2016,RODSILVA2016
122,PRINCESS JACOB,749,0.00,2016,PRINCESSJACOB2016
123,JERRY WHITE,475,0.00,2016,JERRYWHITE2016
124,BRADFORD LYTTLE,382,0.00,2016,BRADFORDLYTTLE2016


<h4>Export Dataframe to CSV and JSON Forat</h4>

In [35]:
election_df.to_csv("popular_votes_years.csv")
election_df.to_json("popular_votes_years.json")

<h2>Store DataFrames into SQL Database Tables</h2>

In [19]:
# conn = sqlite3.connect('ETL_project.db')
# c = conn.cursor()

# c.execute("DROP TABLE IF EXISTS Finance")
# c.execute('CREATE TABLE Finance (candidate_election_year number, candidate_id text, candidate_name text, party_full text, total_receipts number, total_disbursements number, cash_on_hand_end_period number)')
# conn.commit()

# candidate_finance.to_sql('Finance', conn, if_exists='replace', index = False)


# c.execute("SELECT * FROM Finance WHERE candidate_election_year = 2016")

# print("=========================")
# print("Financial Data for 2016")
# for row in c.fetchall():
#     print (row)
# ################################################################
# c.execute("DROP TABLE IF EXISTS Votes")
# c.execute('CREATE TABLE Votes (Name text, Votes int, VotePct text, year int)')
# conn.commit()

# election_df.to_sql('Votes', conn, if_exists='replace', index = False)


# c.execute("SELECT * FROM Votes WHERE year = 2016")

# print("=========================")
# print("Vote Data for 2016")
# for row in c.fetchall():
#     print (row)

Financial Data for 2016
(2016, 'P00003392', 'HILLARY RODHAM CLINTON', 'DEMOCRATIC PARTY', 585699061.27, 585580576.76, 323317.48)
(2016, 'P80001571', 'DONALD J TRUMP', 'REPUBLICAN PARTY', 350668435.7, 343056732.78, 7611702.92)
(2016, 'P60007168', 'BERNARD SANDERS', 'DEMOCRATIC PARTY', 237640609.52, 232185441.62, 5455167.9)
(2016, 'P60006111', 'RAFAEL EDWARD "TED" CRUZ', 'REPUBLICAN PARTY', 94338654.84, 94304803.32, 33851.52)
(2016, 'P60005915', 'BENJAMIN S SR MD CARSON', 'REPUBLICAN PARTY', 65091035.97, 64530285.46, 560750.51)
(2016, 'P60006723', 'MARCO RUBIO', 'REPUBLICAN PARTY', 48331861.99, 51557701.76, 112614.3)
(2016, 'P60008059', 'JEB BUSH', 'REPUBLICAN PARTY', 35491191.48, 35435885.43, 55306.05)
(2016, 'P60003670', 'JOHN R KASICH', 'REPUBLICAN PARTY', 19692691.23, 19534124.57, 158566.66)
(2016, 'P40003576', 'RAND PAUL', 'REPUBLICAN PARTY', 12458992.11, 12452499.19, 6492.92)
(2016, 'P60007242', 'CARLY FIORINA', 'REPUBLICAN PARTY', 12211897.41, 11412309.97, 799587.44)
(2016, 'P2000

<h4>Practice putting SQL back into Dataframe</h4>

In [20]:
# c.execute('''  
# SELECT candidate_name, votes, total_receipts, year
# FROM Finance
# INNER JOIN Votes ON Finance.candidate_name = Votes.Name AND candidate_election_year = year
# WHERE Finance.candidate_election_year = 2016
# LIMIT 10;
#           ''')

# # max_receipts = pd.DataFrame(c.fetchall(), columns=['candidate_name','total_receipts'])    
# results = pd.DataFrame(c.fetchall(), columns=['Name','Votes', 'Total Reciepts', 'Year'])   
# print (results)

             Name       Votes  Total Reciepts  Year
0  DONALD J TRUMP  62,984,828    3.506684e+08  2016
1    GARY JOHNSON   4,489,341    1.219398e+07  2016
2      JILL STEIN   1,457,218    1.149955e+07  2016
3   EVAN MCMULLIN     731,991    1.644102e+06  2016


### Connect to local database

In [106]:
pg_user = 'postgres'
pg_password = '*****'
db_name = 'election_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

### Check for tables

In [107]:
engine.table_names()

['candidate_financial', 'candidate_votes']

### Use Pandas to load DataFrame into Database

In [108]:
candidate_finance.to_sql(name='candidate_financial', con=engine, if_exists='append', index=False)

In [109]:
election_df.to_sql(name='candidate_votes', con=engine, if_exists='append', index=False)

### Confirm Data has been loaded by querying the tables

In [110]:
pd.read_sql_query('select * from candidate_financial', con=engine).head()

,key,candidate_election_year,candidate_id,candidate_name,party_full,total_receipts,total_disbursements,cash_on_hand_end_period
0,GEORGEWBUSH2000,2000,P00003335,GEORGE W BUSH,REPUBLICAN PARTY,193657728.8,1.864560e+08,7201734.18
1,ALGORE2000,2000,P80000912,AL GORE,DEMOCRATIC PARTY,133108037.0,1.203352e+08,12772827.00
2,STEVEFORBES2000,2000,P60003852,STEVE FORBES,REPUBLICAN PARTY,86048573.0,8.605477e+07,0.00
3,JOHNSMCCAIN2000,2000,P80002801,JOHN S MCCAIN,REPUBLICAN PARTY,58636230.0,5.854997e+07,86262.00
4,BILLBRADLEY2000,2000,P80000516,BILL BRADLEY,DEMOCRATIC PARTY,51156171.0,5.042141e+07,775203.00


In [111]:
pd.read_sql_query('select * from candidate_votes', con=engine).head()

,key,name,votes,votepct,year
0,ALGORE2000,AL GORE,50999897,48.38,2000
1,GEORGEWBUSH2000,GEORGE W BUSH,50456002,47.87,2000
2,RALPHNADER2000,RALPH NADER,2882955,2.74,2000
3,PATRICKJBUCHANAN2000,PATRICK J BUCHANAN,448895,0.42,2000
4,HARRYBROWNE2000,HARRY BROWNE,384431,0.36,2000
